In [2]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers import LSTM, Merge
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [10]:
df_input = pd.read_csv("clean_and_gaussian.csv", index_col=0)
df_input_segmented = pd.read_csv("clean_and_gaussian_segmented.csv", index_col=0)
df_concat = pd.concat([df_input, df_input_segmented])

# Shuffle rows so that not all 10s are at the beginning and all 1s at the end
df_concat = df_concat.sample(frac=1)

y = df_concat.iloc[:,-1]
X = df_concat.iloc[:,0:-1]

/Users/Shreya/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [12]:
#train test split 
dataset_size = X.shape[0]
train_size = 7*(dataset_size//10)
train_X = X[0:train_size]
train_y = y[0:train_size].to_frame()
test_X = X[train_size:]
test_y = y[train_size:].to_frame()
print(train_X.shape)
print(type(train_X), train_X.shape)
print(type(train_y), train_y.shape)

(5047, 1198)
<class 'pandas.core.frame.DataFrame'> (5047, 1198)
<class 'pandas.core.frame.DataFrame'> (5047, 1)


In [13]:
import tensorflow as tf
from tensorflow.contrib import rnn

#define constants
#unrolled through 28 time steps
time_steps=1198
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=1
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 11 classes(0-10).
n_classes=11 
#size of batch
batch_size=128

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [14]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [19]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    start_i = 0
    
    while iter<39: #used to be 800
        print('iter ', iter)
        train_X_temp = train_X.iloc[start_i:start_i + 128]
        
        batch_x,batch_y=train_X_temp.values.reshape(1,1198,128), pd.get_dummies(train_y['reps'].values).iloc[start_i: start_i + 128,:]
        
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
    
        sess.run(opt, feed_dict={x: batch_x, y: batch_y.values})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
        start_i = start_i + 128


iter  1
iter  2
iter  3
iter  4
iter  5
iter  6
iter  7
iter  8
iter  9
iter  10
For iter  10
Accuracy  0.4296875
Loss  nan
__________________
iter  11
iter  12
iter  13
iter  14
iter  15
iter  16
iter  17
iter  18
iter  19
iter  20
For iter  20
Accuracy  0.4375
Loss  nan
__________________
iter  21
iter  22
iter  23
iter  24
iter  25
iter  26
iter  27
iter  28
iter  29
iter  30
For iter  30
Accuracy  0.4296875
Loss  nan
__________________
iter  31
iter  32
iter  33
iter  34
iter  35
iter  36
iter  37
iter  38


In [ ]:
#calculating test accuracy
test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
test_label = mnist.test.labels[:128]
print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_X, y: test_y}))

In [38]:
#https://stackoverflow.com/questions/39674713/neural-network-lstm-input-shape-from-dataframe/40005797#40005797

# Extract your training data
train_X['single_input_vector'] = train_X.apply(tuple, axis=1).apply(list)
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
train_X['single_input_vector'] = train_X.single_input_vector.apply(lambda x: [list(x)])
# Use .cumsum() to include previous row vectors in the current row list of vectors
train_X['cumulative_input_vectors'] = train_X.single_input_vector.cumsum()

# Pad your sequences so they are the same length
from keras.preprocessing.sequence import pad_sequences

max_sequence_length = train_X.cumulative_input_vectors.apply(len).max()
# Save it as a list   
padded_sequences = train_X.T
train_X['padded_input_vectors'] = pd.Series(padded_sequences).apply(np.asarray)

X_train_init = np.asarray(train_X)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(train_X),1198,1198)
y_train = np.hstack(np.asarray(train_y)).reshape(len(train_X),1)


/Users/Shreya/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Shreya/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Shreya/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

TypeError: 'int' object is not iterable

In [31]:
#build the model: 2 stacked LSTM
print('Building LSTM model')
hidden_size = 500
model = Sequential()

model.add(LSTM(hidden_size, return_sequences=True, input_shape=(1198,1)))
model.add(Flatten())
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocabulary)))
#model.add(Activation('softmax'))

# left_branch = Sequential()
# left_branch.add(LSTM(512, return_sequences=True, input_shape=(1198,1)))
# right_branch = Sequential()
# right_branch.add(LSTM(512, return_sequences=True, input_shape=(1198,1)))

# model = Sequential()
# model.add(Merge([left_branch, right_branch], mode='concat'))
# model.add(Dropout(0.2)
# model.add(LSTM(512, return_sequences=False))
# model.add(Dropout(0.2))
model.add(Dense(2))

# Softmax activation function
model.add(Activation('softmax'))

# Cross-entropy loss, metric is accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Building LSTM model


ValueError: Input 0 is incompatible with layer lstm_10: expected ndim=3, found ndim=2

In [26]:
batch_size = 50
num_steps = 1
num_epochs = 200
model.fit_generator([train_X, train_y], train_X.shape[0]//(batch_size*num_steps), num_epochs,
                        validation_data=[test_X, test_y],
                        validation_steps=test_X.shape[0]//(batch_size*num_steps))

ValueError: Error when checking input: expected lstm_5_input to have 3 dimensions, but got array with shape (153, 1198)

In [20]:
print(np.asarray(test_X).shape)
print(np.asarray(test_y).shape)

(153, 1198)
(153,)


In [21]:
batch_size = 1000
model.fit([np.asarray(train_X)], train_y, batch_size=batch_size, nb_epoch=50, validation_data=([np.asarray(test_X)], test_y))

/Users/Shreya/anaconda/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[85.62141   , 85.61635   , 85.595     , ...,         nan,
                nan,         nan],
       [97.61356   , 97.61356   , 97.61356   , ...,         nan,
                nan,         nan],...

In [17]:
score, acc = model.evaluate([np.asarray(test_X), np.asarray(test_Y)], test_Z)
print('Test score:', score)
print('Test accuracy:', acc)

11941/11941 [==============================] - 141s 12ms/step
Test score: 0.292373455935
Test accuracy: 0.900510844993


In [18]:
#save our model and data
model.save_weights('model', overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [19]:
evals = model.predict([np.asarray(test_X), np.asarray(test_Y)])
prob_m = [i[0] for i in evals]

In [20]:
out = pd.DataFrame(prob_m)
out['first'] = test['first'].reset_index()['first']
out['last'] = test['last'].reset_index()['last']
out['b_or_n'] = test['b_or_n'].reset_index()['b_or_n']

In [21]:
out.head(10)
out.columns = ['prob_b','first', 'last', 'actual']
out.head(10)
out.to_csv("pred_out.csv")

In [45]:
# small test. first 3 names are spanish and last 3 names are Brazilian 
first = ["manuel","luis","ezequil","michel","luiz","alvaro"]
last = ["garcia","sanchez","gomez","caetano","guilherme","lime"]
X = []
Y = []
trunc_first = [i[0:maxlen] for i in first]
trunc_last = [i[0:maxlen] for i in last]
for i in trunc_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
for i in trunc_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    Y.append(tmp)
    
pred = model.predict([np.asarray(X), np.asarray(Y)])
pred

array([[ 0.41819027,  0.58180976],
       [ 0.36226737,  0.63773263],
       [ 0.61281317,  0.38718686],
       [ 0.85553694,  0.14446311],
       [ 0.99840802,  0.00159197],
       [ 0.98669618,  0.01330382]], dtype=float32)